# ED: Project 2 - Preprocessing

The first step in any Machine Learning is to preprocess the data to obatin a stable and normalized form.
For that reason we aggregated the data in an unique file and normalized it following a Z-Score technique.
Furthermore we added additional features generated from the junction through multiplications of already existing ones.

In [133]:
import os
import pandas as pd
import itertools

dataFolder="../RAVEN/Trial"
datafiles=[dataFolder+"/"+x for x in os.listdir(dataFolder)]
myData=pd.DataFrame()

#getting all info into one file
#mapping wrong question to 0
#mapping right question to 1
for file in datafiles:
    xlsxfile=pd.ExcelFile(file)
    sheets = {sheet_name: xlsxfile.parse(sheet_name) for sheet_name in xlsxfile.sheet_names}
    if file.split("_")[-1].split(".")[0]=="Right":
        answer=1
    else:
        answer=0
    for sheet in sheets:
        if not sheets[sheet].empty:
            subjects=sheets[sheet]["Unnamed: 22"]
            del sheets[sheet]["Unnamed: 22"]
            subjects=subjects.str.startswith("DEI")
            subjects=subjects.astype(int)
            sheets[sheet]["Answer"]=answer
            sheets[sheet]["Area"]=subjects
            myData=pd.concat([myData,sheets[sheet]], ignore_index=True)
            
#for col in myData.columns:
#    if col != "Answer" and col != "Area":
#        if "DISPLAY" in col:
#            name=col.replace("DISPLAY", "").strip()
#            myData[name]=myData[col]*myData[name+" SOLUTIONS"]

#centering and normalizing data
for col in myData:
    if col != "Answer" and col != "Area":
        myData[col]=(myData[col]-myData[col].mean())/myData[col].std()
for col in myData:
    if col != "Answer" and col != "Area":
        myData[col]=myData[col][~((myData[col]-myData[col].mean()).abs() > 3*myData[col].std())]

myData=myData.dropna()            
            
label=myData["Area"]
del myData["Area"]
myData["Area"]=label
myData

,P100 PZ PEAK DISPLAY,P100 PZ LATENCY DISPLAY,P100 CZ PEAK DISPLAY,P100 CZ LATENCY DISPLAY,P300 CZ MEAN AMPLITUDE DISPLAY,P300 PZ MEAN AMPLITUDE DISPLAY,STRESS F3 DISPLAY,STRESS F4 DISPLAY,FATIGUE DISPLAY,LATERALIZATION DISPLAY,...,P100 CZ LATENCY SOLUTIONS,P300 CZ MEAN AMPLITUDE SOLUTIONS,P300 PZ MEAN AMPLITUDE SOLUTIONS,STRESS F3 SOLUTIONS,STRESS F4 SOLUTIONS,FATIGUE SOLUTIONS,LATERALIZATION SOLUTIONS,IMMERSION SOLUTIONS,Answer,Area
0,-0.084678,-1.256915,0.658055,-1.385416,-0.010199,-0.099914,-0.584899,-0.671930,0.306322,0.885628,...,1.450717,0.269095,0.105338,0.219519,-0.072288,-0.202037,-0.732520,0.016837,0,0
1,-0.742726,1.778226,-0.431127,-0.256387,-0.071432,-0.058363,-0.332482,-0.395966,-0.265385,0.146559,...,-1.645850,-0.095970,0.547693,0.534265,0.793597,-0.162977,0.236497,0.400748,0,0
2,-0.298295,0.513584,-0.686758,-1.009073,-0.030254,0.009987,-0.020280,0.399519,-0.227872,0.717974,...,0.335953,0.115636,0.155468,0.194537,0.367470,-0.220096,0.396428,-0.321433,0,0
3,0.182556,1.272369,0.097002,1.248984,-0.024746,0.015734,-0.340009,-0.379528,-0.195151,-0.017331,...,-0.283360,-0.073976,-0.050707,0.847648,2.075948,-0.248073,0.824728,0.755877,0,0
4,-0.024410,1.525298,-0.061592,1.499879,-0.060119,-0.049600,-0.148816,-0.144759,-0.119111,0.191208,...,1.202992,-0.024774,-0.049986,0.242458,0.566472,-0.288178,0.302229,-0.462060,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,-0.169540,1.272369,-0.092195,-0.130940,-0.020564,-0.016169,-0.216554,-0.754092,0.062353,-1.669307,...,-1.274262,0.017342,0.077708,-0.176264,-0.094587,0.052608,0.529948,0.275795,0,1
2070,-0.040080,-0.245201,-0.177595,-0.256387,-0.027847,-0.063516,-0.521559,-0.490436,0.440991,-0.851171,...,-0.283360,-0.158681,-0.166956,-0.382354,-0.285275,0.039292,-1.624074,1.008571,0,1
2071,-0.024884,0.892977,0.037227,0.747193,-0.045420,-0.052452,-0.389580,-0.425662,-0.181634,0.223760,...,-0.159498,-0.074306,-0.047508,-0.335210,-0.386169,-0.170099,0.051226,-0.315132,0,1
2072,-0.059988,1.398834,-0.167964,1.374431,-0.039026,0.023087,-0.327394,-0.310103,-0.252162,0.352167,...,0.707541,0.000738,0.024542,-0.457040,-0.366246,0.025767,0.647128,0.629910,0,1


## Data Visualization

For a better understanding of the data and how the features correlation to each other we implemented a simple scatter matrix that permit a more intuitive take over the existing data.

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(myData, alpha=0.2, figsize=(len(myData.columns), len(myData.columns)))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f6751db60b8>,
      dtype=object)

## Persisting the Dataset

Lastly, to remove the dependencies to the original data and to materialize our dataset, we persisted the processed data to a csv file which facilites the following steps in this study.

In [ ]:
myData.to_csv("ourDataset.csv", index=False)